# Mustererkennung in Funkmessdaten

## Aufgabe 1: Laden der Datenbank in Jupyter Notebook

In [3]:
# imports
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pprint as pp

Wir öffnen die Datenbank und lassen uns die Keys der einzelnen Tabellen ausgeben. 
 

In [ ]:
hdf = pd.HDFStore('../../data/raw/TestMessungen_NEU.hdf')
print(hdf.keys)

## Aufgabe 2: Inspektion eines einzelnen Dataframes

Wir laden den Frame x1_t1_trx_1_4 und betrachten seine Dimension. 

In [ ]:
df_x1_t1_trx_1_4 = hdf.get('/x1/t1/trx_1_4')
print("Rows:", df_x1_t1_trx_1_4.shape[0])
print("Columns:", df_x1_t1_trx_1_4.shape[1])

Als nächstes Untersuchen wir exemplarisch für zwei Empfänger-Sender-Gruppen die Attributzusammensetzung.

In [ ]:
# first inspection of columns from df_x1_t1_trx_1_4
df_x1_t1_trx_1_4.head(5)

Für die Analyse der Frames definieren wir einige Hilfsfunktionen.

In [ ]:
# Little function to retrieve sender-receiver tuples from df columns
def extract_snd_rcv(df):
    regex = r"trx_[1-4]_[1-4]"
    # creates a set containing the different pairs
    snd_rcv = {x[4:7] for x in df.columns if re.search(regex, x)}
    return [(x[0],x[-1]) for x in snd_rcv]

In [ ]:
# Sums the number of columns for each sender-receiver tuple
def get_column_counts(snd_rcv, df):
    col_counts = {}
    for snd,rcv in snd_rcv:
        col_counts['Columns for pair {} {}:'.format(snd, rcv)] = len([i for i, word in enumerate(list(df.columns)) if word.startswith('trx_{}_{}'.format(snd, rcv))])
    return col_counts

In [ ]:
# Analyze the column composition of a given measurement.
def analyse_columns(df):
    df_snd_rcv = extract_snd_rcv(df)
    cc = get_column_counts(df_snd_rcv, df)

    for x in cc:
        print(x, cc[x])
    print("Sum of pair related columns: %i" % sum(cc.values()))
    print()
    print("Other columns are:")
    for att in [col for col in df.columns if 'ifft' not in col and 'ts' not in col]:
        print(att)

In [ ]:
# Analyze the values of the target column.
def analyze_target(df):
    print(df['target'].unique())
    print("# Unique values in target: %i" % len(df['target'].unique()))

Bestimme nun die Spaltezusammensetzung von df_x1_t1_trx_1_4.

In [ ]:
analyse_columns(df_x1_t1_trx_1_4)

Betrachte den Inhalt der "target"-Spalte von df_x1_t1_trx_1_4.

In [ ]:
analyze_target(df_x1_t1_trx_1_4)

Als nächstes laden wir den Frame x3_t2_trx_3_1 und betrachten seine Dimension. 

In [ ]:
df_x3_t2_trx_3_1 = hdf.get('/x3/t2/trx_3_1')
print("Rows:", df_x3_t2_trx_3_1.shape[0])
print("Columns:", df_x3_t2_trx_3_1.shape[1])

Gefolgt von einer Analyse seiner Spaltenzusammensetzung und seiner "target"-Werte.

In [ ]:
analyse_columns(df_x3_t2_trx_3_1)

In [ ]:
analyze_target(df_x3_t2_trx_3_1)

Frage: Was stellen Sie bzgl. der „Empfänger-Nummer_Sender-Nummer“-Kombinationen fest? Sind diese gleich? Welche Ausprägungen finden Sie in der Spalte „target“? 

Antwort: Wir sehen, wenn jeweils ein Paar sendet, hören die anderen beiden Sender zu und messen ihre Verbindung zu den gerade sendenden Knoten (d.h. 6 Paare in jedem Dataframe). Sendet z.B. das Paar 3 1, so misst Knoten 1 die Verbindung 1-3, Knoten 3 die Verbindung 3-1 und Knoten 2 und 4 Verbindung 2-1 und 2-3 bzw. 4-1 und 4-3. Die 10 verschiedenen Ausprägungen der Spalte "target" sind oben zu sehen.

## Aufgabe 3: Visualisierung der Messreihe des Datensatz

Wir visualisieren die Rohdaten mit verschiedenen Heatmaps, um so die Integrität der Daten optisch zu validieren und Ideen für mögliche Features zu entwickeln. Hier stellen wir exemplarisch die Daten von Frame df_x1_t1_trx_1_4 dar.

In [ ]:
vals = df_x1_t1_trx_1_4.loc[:,'trx_2_4_ifft_0':'trx_2_4_ifft_1999'].values

In [ ]:
# one big heatmap
plt.figure(figsize=(14, 12))
plt.title('trx_2_4_ifft')
plt.xlabel("ifft of frequency")
plt.ylabel("measurement")
ax = sns.heatmap(vals, xticklabels=200, yticklabels=20, vmin=0, vmax=1, cmap='nipy_spectral_r')
plt.show()

Wir betrachten wie verschiedene Farbschemata unterschiedliche Merkmale unserer Rohdaten hervorheben.

In [ ]:
# compare different heatmaps
plt.figure(1, figsize=(12,10))

# nipy_spectral_r scheme
plt.subplot(221)
plt.title('trx_2_4_ifft')
plt.xlabel("ifft of frequency")
plt.ylabel("measurement")
ax = sns.heatmap(vals, xticklabels=200, yticklabels=20, vmin=0, vmax=1, cmap='nipy_spectral_r')

# terrain scheme
plt.subplot(222)
plt.title('trx_2_4_ifft')
plt.xlabel("ifft of frequency")
plt.ylabel("measurement")
ax = sns.heatmap(vals, xticklabels=200, yticklabels=20, vmin=0, vmax=1, cmap='terrain')

# Vega10 scheme
plt.subplot(223)
plt.title('trx_2_4_ifft')
plt.xlabel("ifft of frequency")
plt.ylabel("measurement")
ax = sns.heatmap(vals, xticklabels=200, yticklabels=20, vmin=0, vmax=1, cmap='Vega10')

# Wistia scheme
plt.subplot(224)
plt.title('trx_2_4_ifft')
plt.xlabel("ifft of frequency")
plt.ylabel("measurement")
ax = sns.heatmap(vals, xticklabels=200, yticklabels=20, vmin=0, vmax=1, cmap='Wistia')

# Adjust the subplot layout, because the logit one may take more space
# than usual, due to y-tick labels like "1 - 10^{-3}"
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.2)


plt.show()

## Aufgabe 3: Groundtruth-Label anpassen

In [ ]:
# Iterating over hdf data and creating interim data presentation stored in data/interim/testmessungen_interim.hdf
# Interim data representation contains aditional binary class (binary_target - encoding 0=empty and 1=not empty)
# and multi class target (multi_target - encoding 0-9 for each possible class)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

interim_path = '../../data/interim/01_testmessungen.hdf'

def binary_mapper(df):
    
    def map_binary(target):
        if target.startswith('Empty'):
            return 0
        else:
            return 1
    
    df['binary_target'] = pd.Series(map(map_binary, df['target']))
    
    
def multiclass_mapper(df):
    le.fit(df['target'])
    df['multi_target'] = le.transform(df['target'])
    
for key in hdf.keys():
    df = hdf.get(key)
    binary_mapper(df)
    multiclass_mapper(df)
    df.to_hdf(interim_path, key)

hdf.close()

Überprüfe neu beschrifteten Dataframe „/x1/t1/trx_3_1“ verwenden. Wir erwarten als Ergebnisse für 5 zu Beginn des Experiments „Empty“ (bzw. 0) und für 120 mitten im Experiment „Not Empty“ (bzw. 1). 

In [ ]:
hdf = pd.HDFStore('../../data/interim/01_testmessungen.hdf')
df_x1_t1_trx_3_1 = hdf.get('/x1/t1/trx_3_1')
print("binary_target for measurement 5:", df_x1_t1_trx_3_1['binary_target'][5])
print("binary_target for measurement 120:", df_x1_t1_trx_3_1['binary_target'][120])
hdf.close()

## Aufgabe 4: Einfacher Erkenner mit Hold-Out-Validierung

Wir folgen den Schritten in Aufgabe 4 und testen einen einfachen Erkenner.

In [ ]:
from evaluation import *
from filters import *
from utility import *
from features import *

### Öffnen von Hdf mittels pandas

In [ ]:
# raw data to achieve target values
hdf = pd.HDFStore('../../data/raw/TestMessungen_NEU.hdf')

### Beispiel Erkenner

#### Datensätze vorbereiten

In [ ]:
# generate datasets
tst = ['1','2','3']
tst_ds = []

for t in tst:

    df_tst = hdf.get('/x1/t'+t+'/trx_3_1')
    lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]
    
    #df_tst_cl,_ = distortion_filter(df_tst_cl)
    
    groups = get_trx_groups(df_tst)
    df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single, label='target')
    df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single)
    df_p2p = rf_grouped(df_tst, groups=groups, fn=rf_ptp_single) # added p2p feature
    
    df_all = pd.concat( [df_std, df_mean, df_p2p], axis=1 ) # added p2p feature
    
    df_all = cf_std_window(df_all, window=4, label='target')
    
    df_tst_sum = generate_class_label_presence(df_all, state_variable='target')
    
    # remove index column
    df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]
    print('Columns in Dataset:',t)
    print(df_tst_sum.columns)
    
    tst_ds.append(df_tst_sum.copy())

In [ ]:
# holdout validation
print(hold_out_val(tst_ds, target='target', include_self=False, cl='rf', verbose=False, random_state=1))

### Schließen von HDF Store

In [ ]:
hdf.close()

## Aufgabe 5: Eigener Erkenner

Für die Konstruktion eines eigenen Erkenners führen wir die entsprechenden Preprocessing und Mapping Schritte ausgehend von den Roddaten erneut durch und passen diese unseren Bedürfnissen an.# Load hdfs data
hdfs = pd.HDFStore("../../data/raw/henrik/TestMessungen_NEU.hdf")

In [3]:
# Load raw data
hdf = pd.HDFStore("../../data/raw/TestMessungen_NEU.hdf")

In [ ]:
# Check available keys in hdf store
print(hdf.keys)

### Vorverarbeitung

Zuerst passen wir die Groundtruth-Label an, entfernen Zeitstempel sowie Zeilenindices und speichern die resultierenden Frames ab.

In [4]:
hdf_path = "../../data/interim/02_tesmessungen.hdf"

In [10]:
# Mapping groundtruth to 0-empty and 1-not empty and prepare for further preprocessing by
# removing additional timestamp columns and index column
# Storing cleaned dataframes (no index, removed _ts columns, mapped multi classes to 0-empty, 1-not empty)
# to new hdfstore to `data/interim/02_testmessungen.hdf`


dfs = []
for key in hdf.keys():
    df = hdf.get(key)
    #df['target'] = df['target'].map(lambda x: 0 if x.startswith("Empty") else 1)  
    # drop all time stamp columns who endswith _ts
    cols = [c for c in df.columns if not c.lower().endswith("ts")]
    df = df[cols]
    df = df.drop('Timestamp', axis=1)
    df = df.drop('index', axis=1)
    df.to_hdf(hdf_path, key)
hdf.close()

Wir sehen, dass nur noch die 6 x 2000 Messungen für die jeweiligen Paare sowie die 'target'-Werte in den resultierenden Frames enthalten sind.

In [5]:
hdf = pd.HDFStore(hdf_path)
df = hdf.get("/x1/t1/trx_1_2")
df.head()

,trx_1_2_ifft_0,trx_1_2_ifft_1,trx_1_2_ifft_2,trx_1_2_ifft_3,trx_1_2_ifft_4,trx_1_2_ifft_5,trx_1_2_ifft_6,trx_1_2_ifft_7,trx_1_2_ifft_8,trx_1_2_ifft_9,...,trx_4_2_ifft_1991,trx_4_2_ifft_1992,trx_4_2_ifft_1993,trx_4_2_ifft_1994,trx_4_2_ifft_1995,trx_4_2_ifft_1996,trx_4_2_ifft_1997,trx_4_2_ifft_1998,trx_4_2_ifft_1999,target
0,0.057569,0.057136,0.056675,0.056187,0.055674,0.055137,0.054577,0.053996,0.053397,0.052780,...,0.030560,0.030439,0.030293,0.030121,0.029925,0.029705,0.029460,0.029193,0.028902,"Empty_0.0,0.0_0.0,0.0"
1,0.047302,0.046338,0.045374,0.044412,0.043456,0.042508,0.041572,0.040651,0.039749,0.038869,...,0.036571,0.036359,0.036123,0.035864,0.035581,0.035275,0.034946,0.034596,0.034225,"Empty_0.0,0.0_0.0,0.0"
2,0.034838,0.033317,0.031765,0.030187,0.028583,0.026958,0.025314,0.023656,0.021989,0.020319,...,0.031501,0.031202,0.030878,0.030531,0.030161,0.029768,0.029353,0.028916,0.028459,"Empty_0.0,0.0_0.0,0.0"
3,0.053769,0.053513,0.053260,0.053011,0.052767,0.052529,0.052299,0.052078,0.051868,0.051670,...,0.035597,0.035503,0.035381,0.035234,0.035061,0.034861,0.034637,0.034387,0.034113,"Empty_0.0,0.0_0.0,0.0"
4,0.042299,0.041589,0.040935,0.040344,0.039823,0.039379,0.039017,0.038745,0.038566,0.038485,...,0.035293,0.034995,0.034670,0.034319,0.033942,0.033540,0.033112,0.032661,0.032187,"Empty_0.0,0.0_0.0,0.0"


In [6]:
# Step-1 repeating the previous taks 4 to get a comparable base result with the now dropped _ts and index column to improve from
# generate datasets
from evaluation import *
from filters import *
from utility import *
from features import *
    
def prepare_features(c, p):
    tst = ['1','2','3']
    tst_ds = []

    for t in tst:

        df_tst = hdf.get('/x'+c+'/t'+t+'/trx_'+p)
        lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]

        #df_tst_cl,_ = distortion_filter(df_tst_cl)

        df_tst,_ = distortion_filter(df_tst)

        groups = get_trx_groups(df_tst)
        df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single, label='target')
        df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single)

        df_p2p = rf_grouped(df_tst, groups=groups, fn=rf_ptp_single) # added p2p feature

        df_kurt = rf_grouped(df_tst, groups=groups, fn=rf_kurtosis_single)

        df_all = pd.concat( [df_std, df_mean, df_p2p, df_kurt], axis=1 ) # added p2p feature

        df_all = cf_std_window(df_all, window=4, label='target')

        df_all = cf_diff(df_all, label='target')

        df_tst_sum = generate_class_label_presence(df_all, state_variable='target')

        # remove index column
        df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]
        # print('Columns in Dataset:',t)
        # print(df_tst_sum.columns)

        tst_ds.append(df_tst_sum.copy())
        
    return tst_ds

tst_ds = prepare_features(c='1', p='3_1')

In [7]:
# Evaluating different supervised learning methods provided in eval.py
# added a NN evaluator but there are some problems regarding usage and hidden layers
# For the moment only kurtosis and cf_diff are added to the dataset as well as the distortion filter
# Feature selection is needed right now!
for elem in ['rf', 'dt', 'nb' ,'nn','knn']:
    print(elem, ":", hold_out_val(tst_ds, target='target', include_self=False, cl=elem, verbose=False, random_state=1))

rf : (0.68489723318058549, 0.028092668703997854)
dt : (0.6743654559826574, 0.044775120354300674)
nb : (0.72718418207281033, 0.024153318974162501)


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


nn : (0.61148611111757967, 0.15601883361092209)
knn : (0.64177424126303684, 0.021400793616231959)


In [8]:
# extra column features generated and reduced with PCA

from evaluation import *
from filters import *
from utility import *
from features import *
from new_features import *

def prepare_features_PCA_cf(c, p):
    tst = ['1','2','3']
    tst_ds = []

    for t in tst:

        df_tst = hdf.get('/x'+c+'/t'+t+'/trx_'+p)

        lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]

        df_tst,_ = distortion_filter(df_tst)

        groups = get_trx_groups(df_tst)

        df_cf_mean = reduce_dim_PCA(cf_mean_window(df_tst, window=3, column_key="ifft", label=None ).fillna(0), n_comps=10)
        #df_cf_std = reduce_dim_PCA(cf_std_window(df_tst, window=3, column_key="ifft", label=None ).fillna(0), n_comps=10)
        df_cf_ptp = reduce_dim_PCA(cf_ptp(df_tst, window=3, column_key="ifft", label=None ).fillna(0), n_comps=10)
        #df_cf_kurt = reduce_dim_PCA(cf_kurt(df_tst, window=3, column_key="ifft", label=None ).fillna(0), n_comps=10)


        #df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single)
        df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single, label='target')
        df_p2p = rf_grouped(df_tst, groups=groups, fn=rf_ptp_single) # added p2p feature
        df_kurt = rf_grouped(df_tst, groups=groups, fn=rf_kurtosis_single)
        df_skew = rf_grouped(df_tst, groups=groups, fn=rf_skew_single)


        df_all = pd.concat( [df_mean, df_p2p, df_kurt, df_skew], axis=1 ) 

        df_all = cf_std_window(df_all, window=4, label='target')

        df_all = cf_diff(df_all, label='target')

        df_all = reduce_dim_PCA(df_all.fillna(0), n_comps=10, label='target')

        df_all = pd.concat( [df_all, df_cf_mean, df_cf_ptp], axis=1)

        df_tst_sum = generate_class_label_presence(df_all, state_variable='target')

        # remove index column
        df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]

        #print('Columns in Dataset:',t)
        #print(df_tst_sum.columns)


        tst_ds.append(df_tst_sum.copy())
        
    return tst_ds

tst_ds_PCA = prepare_features_PCA_cf(c='1', p='3_1')

In [9]:
# Evaluating different supervised learning methods provided in eval.py
# We can see that the column features have increased F1 score of the classifiers
# Best score for Naive Bayes
for elem in ['rf', 'dt', 'nb' ,'nn','knn']:
    print(elem, ":", hold_out_val(tst_ds_PCA, target='target', include_self=False, cl=elem, verbose=False, random_state=1))

rf : (0.70167962430145014, 0.21734257933619272)
dt : (0.66738417727820487, 0.2124111642523161)
nb : (0.88138734100310501, 0.049978003065224964)
nn : (0.8255193837470074, 0.099580758321414775)
knn : (0.70460302131741093, 0.11468773000515596)


In [10]:
def evaluate_models(ds):
    res = {}
    for elem in ['rf', 'dt', 'nb' ,'nn','knn']: 
        res[elem] = hold_out_val(ds, target='target', include_self=False, cl=elem, verbose=False, random_state=1)
    return res

    
def evaluate_performance(c, p):
    # include a prepare data function?
    ds = prepare_features(c, p)
    return evaluate_models(ds)  


def evaluate_performance_PCA_cf(c, p):
    # include a prepare data function?
    ds = prepare_features_PCA_cf(c, p)
    return evaluate_models(ds)  

In [12]:
config = ['1','2','3','4']
pairing = ['1_2','1_4','2_3','3_1','3_4','4_2']
tst_ds = []

res_all = []
for c in config:
    print("Testing for configuration", c)
    for p in pairing:
        print("Analyse performance for pairing", p)
        res = evaluate_performance(c, p)
        res_all.append(res)
        # TODO draw graph
        for model in res:
            print(model, res[model])

Testing for configuration 1
Analyse performance for pairing 1_2


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.64893729198075112, 0.072661981809761314)
nb (0.72001398839361885, 0.051908004978292642)
knn (0.58415479903993783, 0.078241559930959215)
nn (0.5764874861084508, 0.13044510214826935)
dt (0.67238579801544851, 0.058266923007914605)
Analyse performance for pairing 1_4


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.60208332925071295, 0.089743059261808922)
nb (0.63325283931138598, 0.045517579942638837)
knn (0.54081338730857842, 0.094464769939978793)
nn (0.5720688899326668, 0.13024727627113331)
dt (0.6237008209378716, 0.039594890188088296)
Analyse performance for pairing 2_3


...

/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.56183254166979291, 0.068736610510270288)
nb (0.54602149997504024, 0.053397114686506188)
knn (0.51375346783373532, 0.081594277497857193)
nn (0.55249466766346622, 0.010435389869660231)
dt (0.54140879732826608, 0.033013325039030705)
Analyse performance for pairing 3_4
rf (0.58615728191004679, 0.070742301199853627)
nb (0.61619435933989886, 0.037831221165105573)
knn (0.55687319183243777, 0.088555356653199496)
nn (0.61967849399534358, 0.017252168762122277)
dt (0.55393454236034068, 0.036459474466513167)
Analyse performance for pairing 4_2


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


rf (0.57352049732876009, 0.076094004960706438)
nb (0.64047927288178208, 0.097237388406319195)
knn (0.49949518613057869, 0.092785611026083681)
nn (0.61122087274372217, 0.050765134556721797)
dt (0.55823065133694805, 0.029090367329828379)


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
all_keys = set().union(*(d.keys() for d in res_all))
print(all_keys)
print("results for prepare_features() function")
for key in all_keys:
    print("mean F1 for {}:  {}".format(key, sum(item[key][0] for item in res_all)/len(res_all)))

{'rf', 'knn', 'nb', 'dt', 'nn'}
results for prepare_features() function
mean F1 for rf:  0.6329704112983846
mean F1 for knn:  0.5492569476805765
mean F1 for nb:  0.6571988955686305
mean F1 for dt:  0.6281336930258552
mean F1 for nn:  0.6070024795875362


In [16]:
config = ['1','2','3','4']
pairing = ['1_2','1_4','2_3','3_1','3_4','4_2']
tst_ds = []

res_all_PCA = []
for c in config:
    print("Testing for configuration", c)
    for p in pairing:
        print("Analyse performance for pairing", p)
        res = evaluate_performance_PCA_cf(c, p)
        res_all_PCA.append(res)
        # TODO draw graph
        for model in res:
            print(model, res[model])

Testing for configuration 1
Analyse performance for pairing 1_2
rf (0.57361498741327344, 0.19511283254070527)
nb (0.83613340824479765, 0.07377266612408459)
knn (0.67622660088237607, 0.14440065816769604)
nn (0.66061500026802145, 0.12365896217987189)
dt (0.53796608241999555, 0.17031318987416932)
Analyse performance for pairing 1_4
rf (0.5780539701030456, 0.1427826825837937)
nb (0.78778249984587723, 0.053338977313739704)
knn (0.73447431223254256, 0.045076899710088027)
nn (0.71960041418721865, 0.05953889883646752)
dt (0.55023323892743448, 0.12933580518967475)
Analyse performance for pairing 2_3
rf (0.5536507205482547, 0.16537074209745611)
nb (0.65320688892574819, 0.16826698090686992)
knn (0.75859979487318141, 0.07146612334721171)
nn (0.67078458200278746, 0.080909793740883959)
dt (0.60737565285759043, 0.17491834916148105)
Analyse performance for pairing 3_1
rf (0.70167962430145014, 0.21734257933619272)
nb (0.88138734100310501, 0.049978003065224964)
knn (0.70460302131741093, 0.11468773000515

/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.4354839341333836, 0.018628614018703474)
nb (0.68626772297818484, 0.1535935061523967)
knn (0.57736145322231802, 0.092120212735163895)
nn (0.44989587567503991, 0.076439837040941025)
dt (0.58492892021650544, 0.12376198837821167)
Analyse performance for pairing 2_3


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.50799587032264026, 0.13244864826613725)
nb (0.64769080119858546, 0.20824561004973363)
knn (0.54278857447899276, 0.10557415117505173)
nn (0.51128689233014701, 0.13148626074566644)
dt (0.50082131128161089, 0.14770875912227532)
Analyse performance for pairing 3_1


...

rf (0.47222660989006687, 0.061731258845562624)
nb (0.54765501168541653, 0.027563680704073203)
knn (0.45655957197736302, 0.036712597083316)
nn (0.55746612110284854, 0.11620508051514315)
dt (0.51731959021425267, 0.075006398721573581)
Analyse performance for pairing 4_2
rf (0.48046232809299888, 0.059859709412347667)
nb (0.64689178061844677, 0.11371031125377304)
knn (0.48113167589052885, 0.076752180375769571)
nn (0.52979346915973358, 0.042534765000695383)
dt (0.46913558373448866, 0.080179839638372385)
Testing for configuration 3
Analyse performance for pairing 1_2
rf (0.52144474508140515, 0.1173512877778284)
nb (0.80900589265669909, 0.13278346373581032)
knn (0.63275613458875279, 0.09930908999454742)
nn (0.71692677532956328, 0.076646626381633776)
dt (0.49820151324488232, 0.12334958770259231)
Analyse performance for pairing 1_4
rf (0.76876492758301829, 0.17636423259247064)
nb (0.89458224161238686, 0.068163464308074087)
knn (0.71298937001561014, 0.10330467447240442)
nn (0.78563861047406258, 0

/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.57171602090544849, 0.16292967446571466)
nb (0.87213787475749305, 0.048693820548973067)
knn (0.70652013330486041, 0.047063157026160957)
nn (0.79499094729718489, 0.05616707094027746)
dt (0.53350720746897606, 0.11958506928321476)
Analyse performance for pairing 3_4
rf (0.43303941634736809, 0.17698336450107743)
nb (0.71851371459019886, 0.20623952834642695)
knn (0.66454977687726358, 0.089575447967040311)
nn (0.72607227623329795, 0.17145935027956388)
dt (0.4003409078721934, 0.13837169224260534)
Analyse performance for pairing 4_2
rf (0.61312706257222105, 0.17304741243525254)
nb (0.84148355001418162, 0.073946855079354973)
knn (0.6226648012212106, 0.074957050390310573)
nn (0.67662215371931611, 0.12660482236446507)
dt (0.53113390964388785, 0.1291617375247229)
Testing for configuration 4
Analyse performance for pairing 1_2
rf (0.53025255428552942, 0.088895391244193586)
nb (0.71039303912681373, 0.06043678441515702)
knn (0.6121117431861034, 0.082770141330221092)
nn (0.60170268742495681, 0.09

/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


rf (0.57812478149837099, 0.11055280777313591)
nb (0.74737471099846509, 0.064384000823855453)
knn (0.60401249851996852, 0.098583836780840414)
nn (0.64034547216746585, 0.084730261494552414)
dt (0.60104490418125522, 0.17591095864145129)
Analyse performance for pairing 2_3
rf (0.62498154668144068, 0.13832697309077641)
nb (0.73462114231513309, 0.060205675232210931)
knn (0.59603393022694018, 0.11787066656552406)
nn (0.70609582679597593, 0.068671942654534246)
dt (0.56788827108538531, 0.096451713595125402)
Analyse performance for pairing 3_1
rf (0.59350535879772359, 0.074401891119328184)
nb (0.73991131141209154, 0.037159076489392144)
knn (0.52571692950882865, 0.060733682730789204)
nn (0.51950191445313354, 0.094220999973059646)
dt (0.60960123411195821, 0.077066006733656406)
Analyse performance for pairing 3_4
rf (0.55453507918409939, 0.15642399359376505)
nb (0.83682481635961536, 0.044367645522255125)
knn (0.63586000599705805, 0.11126329174437624)
nn (0.6654976250404081, 0.061421095148139371)
dt

In [18]:
all_keys = set().union(*(d.keys() for d in res_all_PCA))
print(all_keys)
print("results for prepare_features_PCA_cf() function")
for key in all_keys:
    print("mean F1 for {}:  {}".format(key, sum(item[key][0] for item in res_all_PCA)/len(res_all_PCA)))

{'rf', 'knn', 'nb', 'dt', 'nn'}
results for prepare_features_PCA_cf() function
mean F1 for rf:  0.5658984294210225
mean F1 for knn:  0.6341062398202546
mean F1 for nb:  0.7539057578388516
mean F1 for dt:  0.5584301515922564
mean F1 for nn:  0.6634206990718322


## Aufgabe 6: Online Erkenner

### Serialisierung des Models für den Online Predictor
Das zuvor gewählte Model wird serialisiert und in 'models/solution_ueb02' gespeichert damit es beim starten der REST-API geladen werden kann.

In [ ]:
from sklearn.externals import joblib
joblib.dump(res['dt'], '../../models/solution_ueb02/model.plk')

### Starten des online servers
#### Hierzu müssen die Abhängigkeiten Flask, flask_restful, flask_cors installiert sein
The following command starts a flask_restful server on localhost port:5444 which answers json post requests. The server is implemented in the file online.py within the ipynb folder and makes use of the final chosen model.
Requests can be made as post request to http://localhost:5444/predict with a json file of the following format:
{ "row": "features" }
be careful that the sent file is valid json. The answer contains the predicted class.
{ "p_class": "predicted class" }
For now the online predictor only predicts the class of single lines sent to it

In [5]:
# Navigate to notebooks/solution_ueb02 and start the server
# with 'python -m online'

In [ ]:
# Nun werden zeilenweise Anfragen an die REST-API simuliert, jeder valider json request wird mit einer
# json prediction response beantwortet